In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import LLM
llm = LLM(
    model = "gemini/gemini-2.0-flash",
    temperature=0.1
)

In [3]:
original_email = '''looping in Priya and the SDS core team. TAS and PRX decks are updated.
We're targeting Friday for the SDS integration, but this depends on DBX availability and final sign-off.

Please sync with the infra team tomorrow morning if possible. This is high priority due to upcoming audits next week.

Let's finalize the POC by EOD Wednesday. Ping me if any blockers — we can't afford slippage at this stage.
'''


from crewai.tools import BaseTool 

class ReplaceJargonsTool(BaseTool):
    name: str = "Replace Jargons"
    description: str = "Replace Jargons with more specific terms"

    def _run(self, email:str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamp project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standup assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }

        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")

        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations detected."

jt = ReplaceJargonsTool()
print(jt.run(original_email))


Using Tool: Replace Jargons
Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'
Consider replacing 'TAS' with 'technical architecture stack'
Consider replacing 'DBX' with 'client database cluster'
Consider replacing 'SDS' with 'Smart Data Syncer'
Consider replacing 'POC' with 'proof of concept'
Consider replacing 'ping' with 'reach out'


In [4]:
from crewai import Agent, Task, Crew    

email_assistant = Agent(
    role = "Email Assistant Agent",
    goal = "Improve emails and make them sound professional and clear",
    backstory = "A highly experienced communication expert skilled in professional email writing",
    tools = [jt],
    verbose= True,
    llm = llm
)

urgency_detector = Agent(
    role = "Urgency Detector Agent",
    goal = "Evaluate the urgency of the email and rate it as High, Moderate, or Low",
    backstory = "An expert in time-sensitive business communication who identifies urgency levels based on deadlines, scheduling terms, and escalation indicators in email content.",
    verbose = True,
    llm = llm
)



In [5]:
email_task = Task(
    description = f''' Take the following rough email and rewrite it into a professional and polished
    version.
    Expand abbreviation:
    """{original_email}""" ''',
    agent = email_assistant,
    expected_output= " A professional writtent email with proper formatting and content."
)

urgency_task = Task(
    description = '''Review the professionally rewritten email above. 
Your job is to:
- Identify any time-sensitive language like deadlines, date mentions, or urgency signals.
- At the end of the email, append this text:
"Urgency Level: <High / Moderate / Low>"

Do not change the email content. Just add the urgency level in a new line at the end.''',
    agent = urgency_detector,
    expected_output = "Same email with an added line stating urgency level."
)

crew = Crew(
    agents = [email_assistant, urgency_detector],
    tasks = [email_task, urgency_task],
    verbose = True
)

result = crew.kickoff()
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 602fd9d8-60c3-4824-b12d-aad479e55aa5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Task:  Take the following rough email and rewrite it into a professional and polished                          │
│      version.                                                                                                   │
│      Expand abbreviation:                                                                                       │
│      """looping in Priya and the SDS core team. TAS and PRX decks are updated.                                  │
│  We're targeting Friday for the SDS integration, but this depends on DBX availability and final sign-off.       │
│                                                                                                                 │
│  Please sync with the infra team tomorrow morning if possible. This is high priority due to upcoming audits     │
│  next week.                                                                                                     │
│                                                                                                                 │
│  Let's finalize the POC by EOD Wednesday. Ping me if any blockers — we can't afford slippage at this stage.     │
│  """                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

d:\ML Lab\crewai-demo\.venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Assistant Agent                                                                                   │
│                                                                                                                 │
│  Thought: I need to refine the provided email to ensure it is professional, clear, and properly formatted.      │
│  This involves expanding abbreviations, clarifying dependencies, and setting clear expectations for deadlines   │
│  and communication. I will use the Replace Jargons tool to improve clarity.                                     │
│                                                                                                                 │
│  Using Tool: Replace Jargons                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"email\": \"looping in Priya and the SDS core team. TAS and PRX decks are updated. We're targeting Friday   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Urgency Detector Agent                                                                                  │
│                                                                                                                 │
│  Task: Review the professionally rewritten email above.                                                         │
│  Your job is to:                                                                                                │
│  - Identify any time-sensitive language like deadlines, date mentions, or urgency signals.                      │
│  - At the end of the email, append this text:                                                                   │
│  "Urgency Level: <High / Moderate / Low>"                                                                       │
│                                                                                                                 │
│  Do not change the email content. Just add the urgency level in a new line at the end.                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Urgency Detector Agent                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Smart Data Syncer (SDS) Integration and Proof of Concept (POC) Finalization                           │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I am including Priya and the Smart Data Syncer (SDS) core team in this email. The technical architecture       │
│  stack (TAS) and Project Phoenix (internal AI revamp project) decks have been updated.                          │
│                                                                                                                 │
│  We are targeting Friday for the Smart Data Syncer (SDS) integration. However, this is contingent upon the      │
│  client database cluster availability and final approval.                                                       │
│                                                                                                                 │
│  Please coordinate with the infrastructure team tomorrow morning, if possible. This is a high priority due to   │
│  the upcoming audits next week.                                                                                 │
│                                                                                                                 │
│  Let's finalize the proof of concept (POC) by end of day (EOD) Wednesday. Please reach out to me if you         │
│  encounter any obstacles – we cannot afford any delays at this stage.                                           │
│                                                                                                                 │
│  Thank you,                                                                                                     │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│  Urgency Level: High                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 18d7603a-e5ff-4eff-a7c0-c6caa9491af8                                                                     │
│  Agent: Urgency Detector Agent                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 602fd9d8-60c3-4824-b12d-aad479e55aa5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Smart Data Syncer (SDS) Integration and Proof of Concept (POC) Finalization             │
│                                                                                                                 │
│  Dear Team,                                                                                                     │
│                                                                                                                 │
│  I am including Priya and the Smart Data Syncer (SDS) core team in this email. The technical architecture       │
│  stack (TAS) and Project Phoenix (internal AI revamp project) decks have been updated.                          │
│                                                                                                                 │
│  We are targeting Friday for the Smart Data Syncer (SDS) integration. However, this is contingent upon the      │
│  client database cluster availability and final approval.                                                       │
│                                                                                                                 │
│  Please coordinate with the infrastructure team tomorrow morning, if possible. This is a high priority due to   │
│  the upcoming audits next week.                                                                                 │
│                                                                                                                 │
│  Let's finalize the proof of concept (POC) by end of day (EOD) Wednesday. Please reach out to me if you         │
│  encounter any obstacles – we cannot afford any delays at this stage.                                           │
│                                                                                                                 │
│  Thank you,                                                                                                     │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│                                                                                                                 │
│  Urgency Level: High                                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Subject: Smart Data Syncer (SDS) Integration and Proof of Concept (POC) Finalization

Dear Team,

I am including Priya and the Smart Data Syncer (SDS) core team in this email. The technical architecture stack (TAS) and Project Phoenix (internal AI revamp project) decks have been updated.

We are targeting Friday for the Smart Data Syncer (SDS) integration. However, this is contingent upon the client database cluster availability and final approval.

Please coordinate with the infrastructure team tomorrow morning, if possible. This is a high priority due to the upcoming audits next week.

Let's finalize the proof of concept (POC) by end of day (EOD) Wednesday. Please reach out to me if you encounter any obstacles – we cannot afford any delays at this stage.

Thank you,

[Your Name]

Urgency Level: High
